In [1]:
from dataclasses import dataclass
from enum import Enum
import random

## Clases utiles para visualizar los resultados

In [2]:
@dataclass
class Node:
    student_info: list
    calendar: list

class Student(Enum):
    GUSTAVO = 0
    GABRIELA = 1
    JUAN = 2
    LUIS = 3

class Subject(Enum):
    MATH = 0
    HISTORY = 1
    PHYSICS = 2
    CHEMISTRY = 3
    SPANISH = 4
    ENGLISH = 5
    BIOLOGY = 6

class Day(Enum):
    MONDAY = 0
    TUESDAY = 1
    WEDNESDAY = 2

MAX_CLASSES_PER_STUDENT = 3
MAX_CLASSES_PER_DAY = len(Student)

## Inicialización del Estado Inicial

In [13]:
global_subjects = list(Subject)

def pick_subjects():
    """Esta funcion se encarga de asignar materias a los estudiantes de la forma mas diversa posible.
    Esto es importante porque de lo contrario se podria dar el caso de que todos los estudiantes se 
    asignen las mismas materias y no se pueda resolver el problema.
    """
    global global_subjects
    
    if len(global_subjects) == 0:
        global_subjects = list(Subject)
    
    if len(global_subjects) < MAX_CLASSES_PER_STUDENT:
        classes = [global_subjects.pop() for _ in range(len(global_subjects))]
        len_classes = len(classes)
        global_subjects = list(Subject)
        random.shuffle(global_subjects)

        for _ in range(MAX_CLASSES_PER_STUDENT - len_classes):
            classes.append(global_subjects.pop())
    else:
        random.shuffle(global_subjects)
        classes = [global_subjects.pop() for _ in range(MAX_CLASSES_PER_STUDENT)]

    return classes

def do_calendar(student_info):
    calendar = {day: [] for day in Day}

    for student in Student:
        for subject, day in student_info[student]:
            calendar[day].append(subject)

    return calendar

              
def initial_node_initializer():
    """
    Esta funcion se encarga de inicializar las variables de forma que se asegure
    que el problema se pueda resolver. Es decir que todos los estudiantes no se 
    asignen mas materias de las que hay dias; y que no los alumnos no se asignen todos
    exactamente las mismas materias.
    """
    student_info = {}
    

    for student in Student:
        classes = pick_subjects()
        days = random.sample(list(Day), MAX_CLASSES_PER_STUDENT)
        student_info[student] = list(zip(classes, days))

    calendar = do_calendar(student_info)
        
    return Node(student_info, calendar)


## Funciones Utiles para Beam Search

In [12]:
def scoring_function(node: Node):
    """
    Esta funcion se encarga de evaluar la calidad de la solucion propuesta, en este caso la calendarizacion.
    """
    calendar = node.calendar
    repeated_subjects_count = 0

    for day in calendar.keys():        
        days_set = set(calendar[day])        
        if len(days_set) < MAX_CLASSES_PER_DAY:
            repeated_subjects_count +=  MAX_CLASSES_PER_DAY - len(days_set) 

    return repeated_subjects_count


def huristic(node: Node):
    """
    implementar la heuristica para resolver el problema. Permutar los dias de los examenes de los estudiantes
    """
    student_info = node.student_info

    students_selected = random.sample(list(Student), 2)
    student1, student2 = students_selected

    random.shuffle(student_info[student1])
    stud1_class = list(student_info[student1].pop())

    random.shuffle(student_info[student2])
    stud2_class = list(student_info[student2].pop())

    stud1_class_day = stud1_class[1]
    stud2_class_day = stud2_class[1]

    stud1_class[1] = stud2_class_day
    stud2_class[1] = stud1_class_day

    student_info[student1].append(tuple(stud1_class))
    student_info[student2].append(tuple(stud2_class))


    node.student_info = student_info
    node.calendar = do_calendar(student_info)
    return node


## Beam Search Class

In [18]:
class BeamSearch:
    def __init__(self, score_function: callable, heuristic: callable, initial_object,\
                  best_score=0, beam_width=2, max_iterations=100):
        """
        Args:
            score_function (callable): Esta funcion toma como parametro el objeto tipo initial object y retorna un valor numerico
            heuristic (callable): La modificacion que se va a hacer al objeto para generar nuevos objetos
            initial_object (_type_): El objeto del estado inicial
            beam_width (int, optional): Beam width. Defaults to 2.
            max_iterations (int, optional): Cantidad máxima de iteraciones. Defaults to 100.
        """

        # Functions
        self.score_function = score_function
        self.heuristic = heuristic

        # Object
        self.initial_object = initial_object

        # Hyperparameters
        self.beam_width = beam_width
        self.max_iterations = max_iterations
        self.best_score = best_score

        self.best_node = None        
        self.best_object = None

        # C list
        self.C = [(initial_object, self.score_function(initial_object))]


    def _iteration(self):        
        for object_tuple in self.C:
            for _ in range(self.beam_width):
                new_object = self.heuristic(object_tuple[0])
                new_score = self.score_function(new_object)

                self.C.append(tuple((new_object, new_score)))

        # Ordenamos la lista de acuerdo a la funcion de scoring
        self.C = sorted(self.C, key=lambda x: x[1])

        # Nos quedamos con los mejores elementos
        if len(self.C) > self.beam_width:
            self.C = self.C[:self.beam_width]


    def search(self):
        # revisamos que el caso inicial sea el mejor
        if self.score_function(self.initial_object) == self.best_score:
            return self.initial_object, self.best_score

        for _ in range(self.max_iterations):
            self._iteration()

        self.best_object = self.C[0]
        best_object_score = self.best_object[1]

        return self.best_object, best_object_score
    

In [20]:
init = initial_node_initializer()
beam_search = BeamSearch(scoring_function, huristic, init, beam_width=3)
best_object, best_score = beam_search.search()

print(best_object)
print()
print(best_score)

Node(student_info={<Student.GUSTAVO: 0>: [(<Subject.BIOLOGY: 6>, <Day.MONDAY: 0>), (<Subject.HISTORY: 1>, <Day.TUESDAY: 1>), (<Subject.ENGLISH: 5>, <Day.WEDNESDAY: 2>)], <Student.GABRIELA: 1>: [(<Subject.CHEMISTRY: 3>, <Day.WEDNESDAY: 2>), (<Subject.HISTORY: 1>, <Day.MONDAY: 0>), (<Subject.BIOLOGY: 6>, <Day.TUESDAY: 1>)], <Student.JUAN: 2>: [(<Subject.SPANISH: 4>, <Day.WEDNESDAY: 2>), (<Subject.PHYSICS: 2>, <Day.MONDAY: 0>), (<Subject.ENGLISH: 5>, <Day.TUESDAY: 1>)], <Student.LUIS: 3>: [(<Subject.MATH: 0>, <Day.WEDNESDAY: 2>), (<Subject.SPANISH: 4>, <Day.TUESDAY: 1>), (<Subject.MATH: 0>, <Day.MONDAY: 0>)]}, calendar={<Day.MONDAY: 0>: [<Subject.BIOLOGY: 6>, <Subject.HISTORY: 1>, <Subject.PHYSICS: 2>, <Subject.MATH: 0>], <Day.TUESDAY: 1>: [<Subject.HISTORY: 1>, <Subject.BIOLOGY: 6>, <Subject.ENGLISH: 5>, <Subject.SPANISH: 4>], <Day.WEDNESDAY: 2>: [<Subject.ENGLISH: 5>, <Subject.CHEMISTRY: 3>, <Subject.SPANISH: 4>, <Subject.MATH: 0>]})